In [17]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [18]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
274,Low-budget murder mystery about a Public Defen...,negative
748,"OK, Chuck Norris has shown up in many an enter...",negative
765,I first saw Love in Limbo playing late on free...,positive
289,Writer/Director Peter Greenaway cements his ti...,negative
782,Within the first 5 minutes of this movie I kne...,negative


In [19]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [20]:
import nltk
nltk.download('wordnet')
df = normalize_text(df)
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,review,sentiment
274,low budget murder mystery public defender tryi...,negative
748,ok chuck norris shown many entertaining movie ...,negative
765,first saw love limbo playing late free air tv ...,positive
289,writer director peter greenaway cement title h...,negative
782,within first minute movie knew one pick fault ...,negative


In [21]:
df['sentiment'].value_counts()

sentiment
positive    257
negative    243
Name: count, dtype: int64

In [22]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
274,low budget murder mystery public defender tryi...,0
748,ok chuck norris shown many entertaining movie ...,0
765,first saw love limbo playing late free air tv ...,1
289,writer director peter greenaway cement title h...,0
782,within first minute movie knew one pick fault ...,0


In [23]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [26]:
import dagshub
from dotenv import load_dotenv

load_dotenv()
mlflow_tracking_uri = os.getenv("mlflow_tracking_uri")
dagshub_repo_owner = os.getenv("dagshub_repo_owner")
dagshub_repo_name = os.getenv("dagshub_repo_name")

dagshub.init(repo_owner=dagshub_repo_owner, repo_name=dagshub_repo_name, mlflow=True)
mlflow.set_tracking_uri(mlflow_tracking_uri)

mlflow.set_experiment('Logistic Regression Baseline')


2025-11-26 14:08:01,210 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/MohanBisunke/MLOps_text_classification_project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "MohanBisunke/MLOps_text_classification_project"

2025-11-26 14:08:01,219 - INFO - Initialized MLflow to track repo "MohanBisunke/MLOps_text_classification_project"


Repository MohanBisunke/MLOps_text_classification_project initialized!

2025-11-26 14:08:01,222 - INFO - Repository MohanBisunke/MLOps_text_classification_project initialized!


<Experiment: artifact_location='mlflow-artifacts:/0d112c18b9734b4fbcd9d14fca3720f9', creation_time=1764138335530, experiment_id='0', last_update_time=1764138335530, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [27]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")


        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-11-26 14:08:08,181 - INFO - Starting MLflow run...
2025-11-26 14:08:08,557 - INFO - Logging preprocessing parameters...
2025-11-26 14:08:09,644 - INFO - Initializing Logistic Regression model...
2025-11-26 14:08:09,644 - INFO - Fitting the model...
2025-11-26 14:08:09,674 - INFO - Model training complete.
2025-11-26 14:08:09,675 - INFO - Logging model parameters...
2025-11-26 14:08:10,029 - INFO - Making predictions...
2025-11-26 14:08:10,030 - INFO - Calculating evaluation metrics...
2025-11-26 14:08:10,039 - INFO - Logging evaluation metrics...
2025-11-26 14:08:11,462 - INFO - Saving and logging the model...
2025/11/26 14:08:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-11-26 14:08:12,167 - ERROR - An error occurred: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
Traceback (most recent call last):
  File "C:\Users\ACER\AppData\Local\Temp\ipykernel_20864\1443250444.py", line 48, in <

🏃 View run sassy-bear-419 at: https://dagshub.com/MohanBisunke/MLOps_text_classification_project.mlflow/#/experiments/0/runs/487651c4339a4ac3b7ebf85859a73763
🧪 View experiment at: https://dagshub.com/MohanBisunke/MLOps_text_classification_project.mlflow/#/experiments/0
